In [1]:
!python -V

Python 3.10.14


In [13]:
import mlflow
import mlflow.sklearn

# Set the tracking URI to an SQLite database file
mlflow.set_tracking_uri('http://ec2-54-147-138-249.compute-1.amazonaws.com:5000')

# Set the experiment name
mlflow.set_experiment("mlops-zoomcamlp")


<Experiment: artifact_location='s3://test-bucket-mlflow-123/2', creation_time=1726197846122, experiment_id='2', last_update_time=1726197846122, lifecycle_stage='active', name='mlops-zoomcamlp', tags={}>

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import pandas as pd
import pickle
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt

In [14]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-54-147-138-249.compute-1.amazonaws.com:5000'


In [15]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [16]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [17]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [29]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = train_dicts
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = val_dicts

In [19]:
df_train[categorical + numerical].head(1)

,PU_DO,trip_distance
0,43_151,1.01


In [20]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [24]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from mlflow.models.signature import infer_signature
import mlflow.xgboost
from hyperopt.pyll.base import scope
from sklearn.pipeline import Pipeline
def objective(params):
    with mlflow.start_run(nested=True):
        # Log hyperparameters
        mlflow.log_params(params)

        # Create a pipeline with DictVectorizer and XGBoost
        pipeline = Pipeline([
            ('dv', DictVectorizer()),  # Step 1: Vectorize the dictionary data
            ('xgb_model', xgb.XGBRegressor(**params))  # Step 2: Train XGBoost model
        ])

        # Prepare training data
        train_dicts = df_train[categorical + numerical].to_dict(orient='records')
        X_train = train_dicts
        val_dicts = df_val[categorical + numerical].to_dict(orient='records')
        X_val = val_dicts

        # Fit the pipeline
        pipeline.fit(X_train, y_train)

        # Predict on validation set
        y_pred = pipeline.predict(X_val)

        # Calculate RMSE and log the metric
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

        # Log the entire pipeline to MLflow
        signature = infer_signature(X_val, y_pred)
        mlflow.sklearn.log_model(pipeline, "pipeline_model", signature=signature)

        return {'loss': rmse, 'status': STATUS_OK}

In [25]:
space = {
    'max_depth': scope.int(hp.quniform('max_depth', 3, 10, 1)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 1000, 100)),
    'min_child_weight': hp.uniform('min_child_weight', 1, 10),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
}

In [27]:
train_data_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet'
val_data_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet'
with mlflow.start_run(): 
    # Log the developer tag
    mlflow.set_tag("model", "xgboost")
    
    # Log data source URLs as tags
    mlflow.set_tag("train_data_url", train_data_url)
    mlflow.set_tag("val_data_url", val_data_url)

    # Read and log datasets
    mlflow.log_param("train_data_shape", df_train.shape)
    mlflow.log_param("validation_data_shape", df_val.shape)


    # Sample the first few rows of the datasets and log them as artifacts
    df_train.head(5).to_csv("train_sample.csv", index=False)
    df_val.head(5).to_csv("val_sample.csv", index=False)
    mlflow.log_artifact("train_sample.csv")
    mlflow.log_artifact("val_sample.csv")
    
    # Tune the XGBoost model using Hyperopt
    trials = Trials()
    best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=1, trials=trials)

    # Log the best hyperparameters
    mlflow.log_params(best_params)

    # Optionally log the full dataset or additional visualizations

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

2024/09/13 11:29:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run rogue-croc-156 at: http://ec2-54-147-138-249.compute-1.amazonaws.com:5000/#/experiments/2/runs/fe42f4c806ce4fbc8dd0ca201505ee2f.

2024/09/13 11:29:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-54-147-138-249.compute-1.amazonaws.com:5000/#/experiments/2.



100%|██████████| 1/1 [02:49<00:00, 169.12s/trial, best loss: 6.403343604032129]


2024/09/13 11:29:14 INFO mlflow.tracking._tracking_service.client: 🏃 View run colorful-sheep-194 at: http://ec2-54-147-138-249.compute-1.amazonaws.com:5000/#/experiments/2/runs/3d7ff8c67f1444b798ed62a955fa45db.
2024/09/13 11:29:14 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-54-147-138-249.compute-1.amazonaws.com:5000/#/experiments/2.


In [31]:
X_train.shape

AttributeError: 'list' object has no attribute 'shape'

In [30]:
best_params = {
    "colsample_bytree": 0.9447724401254488,
    "learning_rate": 0.2576928125052712,
    "max_depth": 10,
    "reg_alpha": 0.01735745977982242,
    "reg_lambda": 0.04766124633013455,
    "subsample": 0.7076825876246899
}

with mlflow.start_run(nested=True):
    # Log hyperparameters
    mlflow.log_params(best_params)
    
    # Convert data to DMatrix (XGBoost data structure)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    
    # Train model with early stopping
    model = xgb.train(best_params, 
                      dtrain, 
                      evals=[(dval, 'validation')], 
                      early_stopping_rounds=10,
                      num_boost_round=50, 
                      verbose_eval=False)
    
    # Predict on validation set
    y_pred = model.predict(dval)

    # Calculate RMSE and log the metric
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    # Log model signature and input example
    signature = infer_signature(X_val, y_pred)
    
    # Provide an input example (1 sample) for model logging
    input_example = X_val[0:1]  # Adjust if needed for correct format
    mlflow.xgboost.log_model(model, "xgboost_model", signature=signature, input_example=input_example)


2024/09/13 13:25:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run rumbling-moth-324 at: http://ec2-54-147-138-249.compute-1.amazonaws.com:5000/#/experiments/2/runs/6defa097ca3c46268bbe0e9db459903c.
2024/09/13 13:25:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-54-147-138-249.compute-1.amazonaws.com:5000/#/experiments/2.


ValueError: Please reshape the input data into 2-dimensional matrix.

In [22]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
# Start MLflow run
models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(),
    "GradientBoosting": GradientBoostingRegressor(),
}
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        # Enable autologging
        mlflow.sklearn.autolog()

        # Train the model
        model.fit(X_train, y_train)

        # Predict on validation set
        y_pred = model.predict(X_val)

        # Calculate and log RMSE
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)



        print(f"Logged {model_name} with RMSE: {rmse}")

2024/09/06 16:36:18 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


Logged LinearRegression with RMSE: 7.758715203341164


2024/09/06 16:36:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


Logged RandomForest with RMSE: 6.913010836412057


2024/09/06 16:59:31 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


Logged GradientBoosting with RMSE: 6.742303328497425


In [ ]:
import os
import pickle

# Create the 'models' directory if it doesn't exist
if not os.path.exists('models'):
    os.makedirs('models')

# Save the model to a file
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)


In [56]:
import pickle

# Load the model from the file
with open('models/lin_reg.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)


In [58]:
y_pred = lr.predict(X_val[0])

# root_mean_squared_error(y_val, y_pred)

In [59]:
y_pred

array([13.99325864])